### Importing Necessary libraries

In [17]:
import pandas as pd
import numpy as np
import gensim
from gensim.parsing.preprocessing import preprocess_documents
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd


### Scraping the data from NIKE website

In [ ]:
description=[]
link=[]
url = ['https://www.nike.com/in/w/mens-tops-t-shirts-9om13znik1', 
       'https://www.nike.com/in/w/womens-tops-t-shirts-5e1x6z9om13',
      'https://www.nike.com/in/w/hoodies-sweatshirts-5e1x6z6riveznik1', 
       'https://www.nike.com/in/w/jackets-gilets-50r7yz5e1x6znik1',
      'https://www.nike.com/in/w/shorts-38fphz5e1x6znik1',
      'https://www.nike.com/in/w/trousers-tights-2kq19z5e1x6znik1',
      'https://www.nike.com/in/w/womens-skirts-dresses-5e1x6z8y3qp',
      'https://www.nike.com/in/w/womens-modest-wear-22fj5z5e1x6',
      'https://www.nike.com/in/w/womens-back-to-basics-5e1x6zkkjj']
##### Web scrapper for infinite scrolling page #####
for j in url:
    driver = webdriver.Chrome(executable_path=r"C:\Users\Rehan Shaikh\Downloads\chrome driver\chromedriver.exe")
    driver.get(j)
    time.sleep(2)  # Allow 2 seconds for the web page to open
    scroll_pause_time = 1 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
    screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
    i = 1

    while True:
        # scroll one screen height each time
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
        i += 1
        time.sleep(scroll_pause_time)
        # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
        scroll_height = driver.execute_script("return document.body.scrollHeight;")  
        # Break the loop when the height we need to scroll to is larger than the total scroll height
        if (screen_height) * i > scroll_height:
            break 

    ##### Extract DATA  #####

    soup = BeautifulSoup(driver.page_source, "html.parser")
    description += [i.text for i  in soup.find_all('div',class_="product-card__subtitle")]
    link += [a_tag['href'] for a_tag in soup.find_all('a',class_ = 'product-card__img-link-overlay')]
data = {"description":description,"product_link":link}
df = pd.DataFrame(data)
df_final.to_csv('nike.csv',index=False)


### Scraping the data from LEVI's website

In [78]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
# URL of the Levis men's and women's clothing section
url = ['https://www.levi.in/men/clothing/t-shirts-polos?prefn1=items_type&prefv1=Polos%7CT-Shirts&sz=482',
      'https://www.levi.in/men/clothing/pants?sz=118',
      'https://www.levi.in/men/clothing/shorts?sz=57',
      'https://www.levi.in/men/clothing/jackets?sz=72',
      'https://www.levi.in/men/clothing/shirts?sz=504',
      'https://www.levi.in/men/clothing/jeans?sz=612',
      'https://www.levi.in/women/clothing/jeans?sz=569',
      'https://www.levi.in/women/clothing/t-shirts-tops?sz=255',
      'https://www.levi.in/women/clothing/shirts?sz=32',
      'https://www.levi.in/women/clothing/jackets?start=18&sz=18',
      'https://www.levi.in/women/clothing/shorts-skirts?start=18&sz=18',
      'https://www.levi.in/women/clothing/dresses-jumpsuits?sz=31',
      'https://www.levi.in/women/clothing/pants?sz=36']


# Send a GET request to the website

title=[]
description=[]
link=[]
for j in url:
    response = requests.get(j)
    print(response)
    soup = bs(response.text)
    description += [i.text.strip('\n') for i  in soup.find_all('a',attrs={'class': 'name-link'})]
    homepage = 'https://www.levi.in'
    link += [homepage+a_tag['href']for a_tag in soup.find_all('a',attrs={'class': 'name-link'})]
data1 = {"description":description,"product_link":link}
df_levis = pd.DataFrame(data1)


<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


(2789, 2)

#### Merging two dataframe 

In [80]:
df_nike = pd.read_csv('nike.csv')
df_nike.shape

(1015, 2)

In [81]:
cs = df_nike.append(df_levis)
cs.to_csv('clothing_similarity.csv',index=False)

In [82]:
df = pd.read_csv('clothing_similarity.csv')
df.head()

,description,product_link
0,Men's Max90 T-Shirt,https://www.nike.com/in/t/sportswear-max90-t-s...
1,Men's Fitness Tank,https://www.nike.com/in/t/dri-fit-adv-aps-fitn...
2,Short-Sleeve Button-Down Top,https://www.nike.com/in/t/naomi-osaka-short-sl...
3,Men's Basketball T-shirt,https://www.nike.com/in/t/dri-fit-basketball-t...
4,Men's Nike Dri-FIT Mesh Basketball Jersey,https://www.nike.com/in/t/kevin-durant-dri-fit...


####  Preprocessing the data

In [83]:
# Cleaning the description column 
def preprocess(string):
    regex = re.compile("Levi's®")
    return regex.sub('',string).strip(' ')
df_levis['description']=df_levis['description'].apply(preprocess)



def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    
    # Tokenize text into words
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    return tokens

## Building the model

In [89]:

# Preprocess the text corpus
text_corpus = df['description']
processed_corpus = [preprocess_text(doc) for doc in text_corpus]

# Create a dictionary of unique words in the corpus
dictionary = gensim.corpora.Dictionary(processed_corpus)

# Convert the corpus into Bag-of-Words representation
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]

# Compute TF-IDF weights for the corpus
tfidf = gensim.models.TfidfModel(bow_corpus, smartirs='npu')
tfidf_corpus = tfidf[bow_corpus]

# Build an index for similarity calculations
index = gensim.similarities.MatrixSimilarity(tfidf_corpus)

# Define the ranked_results function that accepts a text query and returns the ranked suggestions
def ranked_results(text):
    # Preprocess the new document/query
    new_doc = preprocess_text(text)
    
    # Convert the preprocessed document to Bag-of-Words representation
    new_vec = dictionary.doc2bow(new_doc)
    
    # Compute TF-IDF weights for the new document
    vec_bow_tfidf = tfidf[new_vec]
    
    # Calculate similarities between the new document and the corpus
    sims = index[vec_bow_tfidf]
    
    # Sort the similarities in descending order and select the top 5 results
    sorted_sims = sorted(enumerate(sims), key=lambda item: -item[1])[:5]
    
    # Print the ranked results with their similarity scores
    for idx, similarity in sorted_sims:
        product_link = df['product_link'].iloc[idx]
        print({product_link : round(similarity,2)})

# Calling the ranked_results function with a sample text query
ranked_results("womens running leggings")


{'https://www.nike.com/in/t/epic-fast-mid-rise-running-leggings-L0MXcm/CZ9241-655': 0.8}
{'https://www.nike.com/in/t/epic-fast-mid-rise-running-leggings-L0MXcm/CZ9241-655': 0.8}
{'https://www.nike.com/in/t/epic-fast-mid-rise-running-leggings-L0MXcm/CZ9241-010': 0.8}
{'https://www.nike.com/in/t/air-fast-mid-rise-7-8-running-leggings-drTS5J/DX0299-291': 0.65}
{'https://www.nike.com/in/t/air-fast-mid-rise-7-8-running-leggings-drTS5J/DX0299-291': 0.65}
